# Propriedades das imagens geoespaciais

### Características das bandas 

A página de desenvolvedor do **Google** fornece informações detalhadas sobre o catálogo *Harmonized Sentinel-2 MSI: MultiSpectral Instrument, Level-2A*, assim como os demais. A tabela de referência contendo informações sobre as bandas das imagens pode ser obtida no endereço https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED#bands.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define a URL da página web
url = "https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED#bands"

# Faz uma requisição HTTP para a URL
response = requests.get(url)

# Verifica se a requisição foi bem-sucedida
if response.status_code == 200:
    # Cria um objeto BeautifulSoup para analisar o código fonte HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontra a tabela de bandas usando a classe 'eecat'
    bands_table = soup.find('table', class_='eecat')

    # Se a tabela for encontrada, extraia as informações
    if bands_table:
        # Cria listas para armazenar as informações das bandas
        band_names = []
        band_units = []
        band_min = []
        band_max = []
        band_scale = []
        band_pixel_size = []
        band_wavelength = []
        band_description = []

        # Itera sobre as linhas da tabela
        for row in bands_table.find_all('tr'):
            # Itera sobre as células da linha
            cells = row.find_all('td')
            # Verifica se a linha tem pelo menos 8 células
            if len(cells) >= 8:
                # Extrai as informações das células
                band_names.append(cells[0].text.strip())
                band_units.append(cells[1].text.strip())
                band_min.append(cells[2].text.strip())
                band_max.append(cells[3].text.strip())
                band_scale.append(cells[4].text.strip())
                band_pixel_size.append(cells[5].text.strip())
                band_wavelength.append(cells[6].text.strip())
                band_description.append(cells[7].text.strip())

        # Cria um DataFrame Pandas com as informações das bandas
        bands_df = pd.DataFrame({
            'Name': band_names,
            'Units': band_units,
            'Min': band_min,
            'Max': band_max,
            'Scale': band_scale,
            'Pixel Size': band_pixel_size,
            'Wavelength': band_wavelength,
            'Description': band_description
        })

        # Filtrando as bandas que começam com 'B'
        df = bands_df[bands_df['Name'].str.startswith('B')]

        # Selecionando as colunas desejadas
        df = df[['Name', 'Scale', 'Pixel Size', 'Wavelength', 'Description']]
        # Exibe o DataFrame Pandas
        print(df)
    else:
        print("A tabela com a classe 'eecat' não foi encontrada na página web.")
else:
    print("Erro ao acessar a página web:", response.status_code)

### Propriedades gerais

Embora nem todas as características das bandas descritas na tabela de referência possam ser acessadas diretamente, diversas propriedades das imagens podem ser exibidas através de métodos fornecidos pelo **Google Earth Engine**, para extração dos metadados.

Para acesso ao catálogo de imagens geoespaciais do **Google Earth Engine**, repetimos os procedimentos descritos anteriormente 

### Importamos as bibliotecas

In [ ]:
import ee
import geemap
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

### Inicializamos a API

In [ ]:
# Processo de autenticação
ee.Authenticate()

# Inicializamos a biblioteca
ee.Initialize()

### Definição de parâmetros

*Intervalo temporal*

In [ ]:
data_inicial = '2024-04-01'
data_final = '2024-07-31'

*Área de interesse*

In [ ]:
polygon_coords = [
    [-43.41287287680422, -21.756108688468274],
    [-43.396307553928246, -21.756108688468274],
    [-43.396307553928246, -21.750887131860495],
    [-43.41287287680422, -21.750887131860495],
    [-43.41287287680422, -21.756108688468274]
]

aoi = ee.Geometry.Polygon(polygon_coords, None, False)

### Escolha da coleção de imagens

Aplicamos os filtros de acordo com os parâmetros:

In [ ]:
db = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(aoi) \
                .filterDate(ee.Date(data_inicial), ee.Date(data_final)) 

### Contagem das imagens disponíveis

In [ ]:
dbCount = db.size()
print('Quantidade: ', str(dbCount.getInfo())+'\n')

### Extraímos a imagem com menor cobertura de nuvens

In [ ]:
db = ee.Image(db.sort('CLOUDY_PIXEL_PERCENTAGE').first())

### Contagem das bandas disponíveis

In [ ]:
# Obtém os nomes das bandas
band_names = db.bandNames()

# Conta o número de bandas
number_of_bands = band_names.size().getInfo()

# Imprime o número de bandas
print(f'Número de bandas: {number_of_bands}')

### Bandas que iniciam com a letra 'B'

In [ ]:
# Leitura das bandas, que começam com "B"
db_bands = db.select('[B].*')

# Criamos um dicionário das bandas
db_band_info = db_bands.getInfo()
type(db_band_info)

# Iteramos sobre o dicionário 
print('Bandas:\n')
for i in range(len(db_band_info['bands'])):
    print(db_band_info['bands'][i]['id'])

### Extraímos as propriedades das bandas selecionadas

In [ ]:
db_bands.getInfo()['bands']

## Metadados

Para obter os metadados no **Google Earth Engine**, chamamos o método *getInfo* no objeto *Image* criado.

In [ ]:
db_info = db.getInfo()

## Listamos o dicionário de metadados associados à imagem

In [ ]:
print("Fields of Image Info:")
for key in db_info:
    print(key)

### Listamos as propriedades da imagem

In [ ]:
properties = db_info.get('properties', {})

print("Principais propriedades da imagem:")
for i, (key, value) in enumerate(properties.items()):
    #if i >= 20:
        #break
    print(f"{key}: {value}")


### Acessamos uma propriedade específica

In [ ]:
# Acessar as propriedades da imagem
properties = db_info.get('properties', {})

# Especificar a propriedade que você quer obter
cloud_shadow_percentage = properties.get('CLOUD_SHADOW_PERCENTAGE')

# Imprimir o valor da propriedade
print(f"PERCENTUAL DE NUVENS: {cloud_shadow_percentage}")


### Listamos a escala das bandas iniciadas com a letra 'B'

In [ ]:
# Obter a lista de bandas
band_names = db.bandNames().getInfo()

for band in band_names:
    if band.startswith('B'):
        scale = db.select(band).projection().nominalScale()
        print(f'Banda {band} Escala: {scale.getInfo()} metros')

### Escala x tamanho do pixel

A escala nominal e o tamanho do *pixel* estão relacionados, mas não são exatamente a mesma coisa. A escala nominal refere-se à resolução espacial da banda em termos de tamanho de *pixel* no solo, mas é expressa como um valor em metros. O tamanho do *pixel* refere-se ao tamanho físico do *pixel* na imagem, geralmente também expresso em metros. 

No entanto, na prática, a escala nominal normalmente coincide com o tamanho do *pixel* em imagens de satélite, especialmente em projeções regulares como **UTM (Universal Transverse Mercator)**, sistema de coordenadas geográficas que divide o mundo em uma série de zonas longitudinais e projeta a superfície terrestre em um plano 2D, utilizando a projeção cilíndrica transversal de *Mercator*.

In [ ]:
# Iterar sobre as bandas que começam com 'B' e obter suas características
for band in band_names:
    if band.startswith('B'):
        scale = db.select(band).projection().nominalScale().getInfo()
        pixel_size = db.select(band).projection().getInfo()['transform'][0]
        print(f'Banda: {band}, Escala: {scale} metros, Tamanho do Pixel: {pixel_size} metros')